# Importing data

In [1]:
Toolbox for EOG et EMG -> Voir si y a des params importants
Faire du ML sur ces features et voir si y a des features importantes pr ça 

SyntaxError: invalid syntax (3358490829.py, line 1)

In [1]:
from glob import glob
import os, mne, numpy as np, matplotlib.pyplot as plt, pandas as pd
import re
from tqdm import tqdm

In [2]:
#Your path to the data
main_path = "../BCI_Database/Signals/DATA A/*"

#### Storing all the attributes of the subjects into a dataframe in order to separate them according to specific conditions

In [3]:
def df_attributes():
    
    #Retrieving attributes of all participants so we can separate them based on specific conditions
    attributes_path = glob("../BCI_Database/*")[3]
    print("Path :", attributes_path)

    attributes = pd.read_excel(attributes_path, header = None)

    attributes.columns = attributes.iloc[2]
    attributes = attributes.iloc[3:63].reset_index(drop=True)

    names = attributes.columns.tolist()
    print("Columns :", names)

    for col in attributes.columns:
        if attributes[col].isnull().sum() !=0:
            print(col, attributes[col].isnull().sum())

    #Using a dictionnary to fill NA values for every column that contains Na
    D = {None : "Nothing/Unknown"}
    attributes = attributes.replace(D)   
    
    #Reject every participant that had some issues in the EEG data acquisition
    rejected = attributes[attributes['COMMENTS'] != "Nothing/Unknown"]['SUJ_ID']
    rejected = rejected.tolist()


    attributes = attributes[attributes['COMMENTS'] == "Nothing/Unknown"]
    
    attributes.to_csv('../attributes.csv', index=False)
    
    return attributes, rejected

In [4]:
attributes, rejected = df_attributes()
display(attributes.head())
print(rejected)

Path : ../BCI_Database\Perfomances.xlsx
Columns : ['SUJ_ID', 'SUJ_gender', 'EXP_gender', 'COMMENTS', 'Perf_RUN_3', 'Perf_RUN_4', 'Perf_RUN_5', 'Perf_RUN_6', 'Birth_year', 'Vision', 'Vision_assistance', 'Symptoms', 'Level of study', 'Level_knowledge neuro', 'Meditation practice', 'Laterality answered', 'Manual activity', 'Manual activity_TXT', 'score', 'time_1', 'time_2', 'PRE_Mood', 'PRE_Mindfulness', 'PRE_Motivation', 'PRE_Hours_sleep_last_night', 'PRE_Usual_sleep', 'PRE_Level_of_alertness', 'PRE_Stimulant_doses_12h', 'PRE_Stimulant_doses_2h', 'PRE_Stim_normal', 'PRE_Tabacco', 'PRE_Tabacco_normal', 'PRE_Alcohol', 'PRE_Last_meal', 'PRE_Last_pills', 'PRE_Pills_TXT', 'POST_Mood', 'POST_Mindfulness', 'POST_Motivation', 'POST_Cognitive load', 'POST_Agentivity', 'POST_Expectations_filled', 'active', 'reflexive', 'sensory', 'intuitive', 'visual', 'verbal', 'sequential', 'global', 'A', 'B', 'C', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N', 'O', 'Q1', 'Q2', 'Q3', 'Q4', 'IM', 'EX', 'AX', 'TM', 'IN',

C:\Users\gricih01\AppData\Local\Temp\ipykernel_18364\977391799.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  attributes = attributes.replace(D)


2,SUJ_ID,SUJ_gender,EXP_gender,COMMENTS,Perf_RUN_3,Perf_RUN_4,Perf_RUN_5,Perf_RUN_6,Birth_year,Vision,...,Q2,Q3,Q4,IM,EX,AX,TM,IN,SC,Interrogation
0,A1,1,1,Nothing/Unknown,67.5,92.5,95,82.5,1993,1,...,7,3,8,5,4.7,9.1,3.1,3.6,3.0,15
2,A3,2,1,Nothing/Unknown,100.0,95.0,97.5,100,1969,2,...,6,3,5,7,5.5,4.2,4.8,3.3,4.0,7
3,A4,1,1,Nothing/Unknown,55.0,45.0,55,50,1982,2,...,7,2,7,3,5.8,6.8,5.0,4.9,2.4,8
6,A7,2,1,Nothing/Unknown,62.5,42.5,45,40,1997,1,...,5,5,3,6,7.7,5.0,7.8,4.7,5.6,4
7,A8,1,2,Nothing/Unknown,50.0,57.4,60,57.5,1992,2,...,6,5,7,2,6.3,9.7,4.1,4.6,4.4,9


['A2', 'A5', 'A6', 'A9', 'A11', 'A13', 'A17', 'A21', 'A29', 'A31', 'A32', 'A36', 'A38', 'A39', 'A46', 'A47', 'A55', 'A60']


In [5]:
#We also delete A40 and A59 because of the missing data (see on the article)
rejected.append("A40")
rejected.append("A59")

This code will create dataframes depending on input files which can be :   
- Acquisition only  
- OnlineT only  
- Acquisition + training

In [6]:
def files(main_path, baseline = False, acquisition = False, MI = True, rejected = []):
    
    #Copying the list since we will remove some subjects in the list
    reject = rejected.copy()

    single_digit = []
    for suj in rejected:
        
        if len(suj) == 2:
            single_digit.append(suj)
            reject.remove(suj)
        
        #Win some time cause 2 digits subjects are after single digit ones
        else:
            break
    
    all_file_path = glob(main_path)

    print(f"Number of participants : {len(all_file_path)}\n")

    all_file_path = [path for path in all_file_path if not any(substring in path.split('DATA')[1] for substring in reject)]
    
    #Remove A1 because of the acquisition which is shorter than other acquisition files
    all_file_path.remove("../BCI_Database/Signals/DATA A\\A1")

    #Removing the subject with single digit
    for path in single_digit:
        all_file_path.remove(f"../BCI_Database/Signals/DATA A\\{path}")

    print("Number of participants retained :" , len(all_file_path))

    all_baseline_path = []
    if baseline :
        all_baseline_path = []
        for path in all_file_path:
            path = glob(path + '/*baseline*')
            all_baseline_path.append(path)
    
    all_acquisition_path = []
    if acquisition :
        for path in all_file_path:
            path = glob(path + '/*acquisition*')
            all_acquisition_path.append(path)

    all_MI_path = []
    if MI :
        for path in all_file_path:
            path = glob(path + '/*online*')
            all_MI_path.append(path)

    if baseline:
        n = len(all_baseline_path)
        
    elif acquisition:
        n = len(all_acquisition_path)

    elif MI:
        n = len(all_MI_path)

    #Creating final list with empty sublists
    L = [[] for _ in range(n)]

    for i in range(n):

        if baseline:
            L[i].extend(all_baseline_path[i])

        if acquisition:
            L[i].extend(all_acquisition_path[i])

        if MI:
            L[i].extend(all_MI_path[i])
    
    #Find participants selected
    pattern = r'A\d+'

    return L

In [7]:
all_files = files(main_path, baseline=False, acquisition=True, MI=True, rejected=rejected)
print(f"\nNumber of files per participant : {len(all_files[0])}\nFiles for participant A10 : {all_files[0]}")

Number of participants : 60

Number of participants retained : 39

Number of files per participant : 6
Files for participant A10 : ['../BCI_Database/Signals/DATA A\\A10\\A10_R1_acquisition.gdf', '../BCI_Database/Signals/DATA A\\A10\\A10_R2_acquisition.gdf', '../BCI_Database/Signals/DATA A\\A10\\A10_R3_onlineT.gdf', '../BCI_Database/Signals/DATA A\\A10\\A10_R4_onlineT.gdf', '../BCI_Database/Signals/DATA A\\A10\\A10_R5_onlineT.gdf', '../BCI_Database/Signals/DATA A\\A10\\A10_R6_onlineT.gdf']


In [8]:
baseline_files = files(main_path, baseline=True, acquisition=False, MI=False, rejected=rejected)
print(f"\nNumber of files per participant : {len(baseline_files[0])} \nFiles for participant A10 : {baseline_files[0]}")

Number of participants : 60

Number of participants retained : 39

Number of files per participant : 2 
Files for participant A10 : ['../BCI_Database/Signals/DATA A\\A10\\A10_CE_baseline.gdf', '../BCI_Database/Signals/DATA A\\A10\\A10_OE_baseline.gdf']


In [16]:
#Reading data
def read_data(file_path):

    # Excluding non EEG channels
    electrodes_to_exclude = ['EOG1', 'EOG2', 'EOG3', 'EMGg', 'EMGd']

    # Importing data
    data = mne.io.read_raw_gdf(file_path, exclude=electrodes_to_exclude, preload=True)

    #We do not set_eeg_reference since the referencing have been already done on the earlobe and reference electrode is not included in the data

    #Doing the minimum filtering (we're still on raw data but we filter out the drifting of EEG)
    data.filter(l_freq=0.5, h_freq=50, method='iir')   #Forced to use bandpass filter so we have 50 as max frequency

    #Creating events based on the article annotation (768 = begin of the trial, 769 = left-hand MI, 770 = right-hand MI)
    events = mne.events_from_annotations(data, event_id={'768': 0, '769': 1, '770': 2})[0]

    #Creating epochs for every trial
    epochs = mne.Epochs(data, events, event_id={'start': 0}, tmin=0, tmax=8, baseline=(0, 2), preload=True) #Setting the baseline to the first two seconds of each trial (Fig.5 of the article)
    
    array = epochs['start'].get_data(copy=True)

    #Retrieving the sampling rate
    global sfreq
    sfreq = int(data.info['sfreq'])

    #Retrieving channel names
    global channel_names
    channel_names = epochs.ch_names

    print(f"\nSampling rate : {sfreq}")
    return array, events

In [20]:
sample_data, events = read_data(all_files[0][0])

Extracting EDF parameters from c:\Users\gricih01\OneDrive - Université du Québec en Outaouais\Documents\MI BCI Database and scripts\Scripts\A5_R1_acquisition.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
Fz, FCz, Cz, CPz, Pz, C1, C3, C5, C2, C4, C6, F4, FC2, FC4, FC6, CP2, CP4, CP6, P4, F3, FC1, FC3, FC5, CP1, CP3, CP5, P3
Creating raw.info structure...
Reading 0 ... 230431  =      0.000 ...   450.061 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 50.00 Hz: -6.02, -6.02 dB

Used Annotations descriptions: ['768', '769', '770']
Not setting metadata
40 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Usin

In [93]:
print(sample_data.shape) #nb of windows, channels, nb of points for each signal
print(events.shape)
del sample_data
del events

(40, 27, 4097)
(80, 3)


In [94]:
def read_baseline(file_path):

    # Excluding non EEG channels
    electrodes_to_exclude = ['EOG1', 'EOG2', 'EOG3', 'EMGg', 'EMGd']

    eog_channels = ['EOG1', 'EOG2', 'EOG3']
    
    # Importing data
    data = mne.io.read_raw_gdf(file_path, eog=eog_channels, exclude=electrodes_to_exclude, preload=True)

    #We do not set_eeg_reference since the referencing have been already done on the earlobe and reference electrode is not included in the data

    #Doing the minimum filtering (we're still on raw data but we filter out the drifting of EEG)
    data.filter(l_freq=0.5, h_freq=50, method='iir', n_jobs=-1)   #Power line noise = 50Hz (Verified on graph + classic Europpean noise line)

    #Creating epochs (same # of epochs as the old one)
    epochs = mne.make_fixed_length_epochs(data, duration = 4.6, overlap = 0)    

    array = epochs.get_data(picks='eeg', copy=True)
    
    return array

In [96]:
baseline_sample = read_baseline(baseline_files[0][0])
print(f'\n\n{baseline_sample.shape}')
#del baseline_sample

Extracting EDF parameters from c:\Users\gricih01\OneDrive - Université du Québec en Outaouais\Documents\MI BCI Database and scripts\BCI_Database\Signals\DATA A\A10\A10_CE_baseline.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
Fz, FCz, Cz, CPz, Pz, C1, C3, C5, C2, C4, C6, F4, FC2, FC4, FC6, CP2, CP4, CP6, P4, F3, FC1, FC3, FC5, CP1, CP3, CP5, P3
Creating raw.info structure...
Reading 0 ... 95775  =      0.000 ...   187.061 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 50.00 Hz: -6.02, -6.02 dB



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 2355 original time points ...
0 bad epochs dropped

(40, 27, 2355)


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    9.5s finished


In [159]:
print(f"Names of the channels : {channel_names}")

Names of the channels : ['Fz', 'FCz', 'Cz', 'CPz', 'Pz', 'C1', 'C3', 'C5', 'C2', 'C4', 'C6', 'F4', 'FC2', 'FC4', 'FC6', 'CP2', 'CP4', 'CP6', 'P4', 'F3', 'FC1', 'FC3', 'FC5', 'CP1', 'CP3', 'CP5', 'P3']


In [160]:
def read_all_data(all_files):

    #Label array to store every ID for each participant
    label_subject_acquisition = []
    label_subject_MI = []

    #Regex pattern to find the ID of the subject
    pattern = r'A\d+'

    data_acquisition_list = []  #Metadata of the acquisition
    data_MI_list = []           #Metadata of the MI

    events_acquisition = []            #Storing the events for every signal
    events_MI = []            #Storing the events for every signal

    for subject_files in all_files:
        
        #Storing acquisition for each subject (each signal selected)
        data_subject_acquisition = []

        #Storing MI for each subject
        data_subject_MI = []
        label = re.findall(pattern, subject_files[0])[0]

        for j in subject_files:

            data, event = read_data(j)

            if 'acquisition' in j:
                data_subject_acquisition.append(data)
                events_acquisition.append(event)

            elif 'onlineT' in j:
                data_subject_MI.append(data)
                events_MI.append(event)

        
        #Creating the array and reshaping it (#files, # epochs, #channels, resolution) -> (#epochs * #files, #channels, #resolution)
        data_subject_acquisition = np.array(data_subject_acquisition)

        #Reshaping acquisition matrix
        files_acquisition, epochs_acquisition, channels_acquisition, resolution_acquisition = data_subject_acquisition.shape
        data_subject_acquisition = data_subject_acquisition.reshape((files_acquisition*epochs_acquisition, channels_acquisition, resolution_acquisition))
        
        #Adding the labels to the arrays
        label_subject_acquisition.append([label] *  (files_acquisition*epochs_acquisition))

        #Adding it to the metadata
        data_acquisition_list.append(data_subject_acquisition)


        #Same for motor imagery
        data_subject_MI = np.array(data_subject_MI)

        #Reshaping motor imagery matrix
        files_MI, epochs_MI, channels_MI, resolution_MI = data_subject_MI.shape
        data_subject_MI = data_subject_MI.reshape((files_MI*epochs_MI, channels_MI, resolution_MI))

        #Adding the labels to the arrays
        label_subject_MI.append([label] *  (files_MI*epochs_MI))

        #Adding it to the metadata
        data_MI_list.append(data_subject_MI)
        
    #Creating the array and reshaping it to the good shape
    events_acquisition = np.array(events_acquisition)
    events_acquisition = events_acquisition.reshape(-1, 3)

    events_MI = np.array(events_MI)
    events_MI = events_MI.reshape(-1, 3)

    return np.array(data_acquisition_list), np.array(data_MI_list), np.array(label_subject_acquisition), np.array(label_subject_MI), events_acquisition, events_MI

In [161]:
%%capture
#Execution takes around 6mins
data_array_acquisition, data_array_MI, label_array_acquisition, label_array_MI, events_acquisition, events_MI = read_all_data(all_files)

In [162]:
print(f"Shape of the acquisition array {data_array_acquisition.shape}") # (# subjects, # windows per subject, # channels, resolution of the signal)
print(f"Shape of its label array {label_array_acquisition.shape}\n")    # (# subjects, # windows per subject)


print(f"Shape of the acquisition array {data_array_MI.shape}") # (# subjects, # windows per subject, # channels, resolution of the signal)
print(f"Shape of its label array {label_array_MI.shape}")

Shape of the acquisition array (39, 80, 27, 4097)
Shape of its label array (39, 80)

Shape of the acquisition array (39, 160, 27, 4097)
Shape of its label array (39, 160)


In [163]:
print(f"Events acquisition shape : {events_acquisition.shape}")
print(f"Events motor imagery shape : {events_MI.shape}")

Events acquisition shape : (6240, 3)
Events motor imagery shape : (12480, 3)


In [164]:
def read_all_baseline(all_baselines):

    #Label array to store every ID for each participant
    label_subject_baseline = []

    #Regex pattern to find the ID of the subject
    pattern = r'A\d+'

    #Same function as before but for baseline files
    data_baseline_list = []

    for subject_files in all_baselines:

        booleen = False
        data_subject_baseline = []
        label = re.findall(pattern, subject_files[0])[0]
        
        for j in subject_files:
                        
            data = read_baseline(j)
            data_subject_baseline.append(data)

            if data.shape[0] !=40:
                booleen = True

        #If the shape is different (for some reasons) we do not inclue the subject in the data
        if booleen:
            continue

        #Creating the array and reshaping it (#files, # epochs, #channels, resolution) -> (#epochs * #files, #channels, #resolution
        data_subject_baseline = np.array(data_subject_baseline)

        #Reshaping it
        files, epochs, channels, resolution = data_subject_baseline.shape
        data_subject_baseline = data_subject_baseline.reshape((files * epochs, channels, resolution))
        
        #Annotation of every subject
        label_subject_baseline.append([label] *  (files*epochs))

        #Adding it to the metadata
        data_baseline_list.append(data_subject_baseline)


    return np.array(data_baseline_list), np.array(label_subject_baseline)

In [165]:
%%capture
data_array_baseline, label_array_baseline = read_all_baseline(baseline_files)

In [166]:
print(f"Shape of the baseline array {data_array_baseline.shape}") # (# subjects, # windows per subject, # channels, resolution of the signal)
print(f"Shape of its label array {label_array_baseline.shape}\n")

Shape of the baseline array (37, 80, 27, 2355)
Shape of its label array (37, 80)



In [167]:
participants_baseline = np.unique(label_array_baseline)

participants = np.unique(label_array_acquisition)

absents = []
for subject in participants:
    if subject not in participants_baseline:
        absents.append(subject)

print(f"Subjects absents in the participants baseline due to some issues : {absents}")

Subjects absents in the participants baseline due to some issues : ['A16', 'A30']


In [168]:
print(f"Baseline number of trials per participant : {label_array_baseline.shape[1]}")
print(f"Acquisition number of trials per participant : {label_array_acquisition.shape[1]}")
print(f"Motor Imagery number of trials per participant : {label_array_MI.shape[1]}")

Baseline number of trials per participant : 80
Acquisition number of trials per participant : 80
Motor Imagery number of trials per participant : 160


In [169]:
label_array_baseline = np.hstack(label_array_baseline)
label_array_acquisition = np.hstack(label_array_acquisition)
label_array_MI = np.hstack(label_array_MI)

print(f"Baseline : {label_array_baseline.shape}")
print(f"Acquisition : {label_array_acquisition.shape}")
print(f"Motor Imagery {label_array_MI.shape}")

Baseline : (2960,)
Acquisition : (3120,)
Motor Imagery (6240,)


In [170]:
#Create the storing folder if it does not exists
if not os.path.exists('../Labels_arrays'):
    os.makedirs('../Labels_arrays')

if not os.path.exists('../Labels_arrays/Raw'):
    os.makedirs('../Labels_arrays/Raw')

data_array_baseline = np.vstack(data_array_baseline)

#Saving the label for analysis
np.savetxt('../Labels_arrays/Raw/Raw_label_array_baseline.txt', label_array_baseline, fmt='%s', delimiter=',')

print(data_array_baseline.shape, label_array_baseline.shape)



data_array_acquisition = np.vstack(data_array_acquisition)   #concatenate all lists vertically

#Saving the label for analysis
np.savetxt('../Labels_arrays/Raw/Raw_label_array_acquisition.txt', label_array_acquisition, fmt='%s', delimiter=',')

print(data_array_acquisition.shape, label_array_acquisition.shape)



data_array_MI = np.vstack(data_array_MI)   #concatenate all lists vertically

#Saving the label for analysis
np.savetxt('../Labels_arrays/Raw/Raw_label_array_MI.txt', label_array_MI, fmt='%s', delimiter=',')

print(data_array_MI.shape, label_array_MI.shape)

(2960, 27, 2355) (2960,)
(3120, 27, 4097) (3120,)
(6240, 27, 4097) (6240,)


In [171]:
windows_baseline, resolution_baseline = data_array_baseline.shape[0], data_array_baseline.shape[2]

windows_acquisition, channels, resolution = data_array_acquisition.shape

windows_MI = data_array_MI.shape[0]

# Feature extraction

In [172]:
#Function to save dataframes/array into csv files
def save_as_csv(X, filename):
    
    #Create the storing folder if it does not exists
    if not os.path.exists('../Features_Dataframes'):
        os.makedirs('../Features_Dataframes')

    if not os.path.exists('../Features_Dataframes/Raw'):
        os.makedirs('../Features_Dataframes/Raw')

    X.to_csv(f'../Features_Dataframes/Raw/{filename}.csv', index=False)
    print(f"{filename} saved as .csv")
    pass

#### PSD features

In [173]:
from scipy.integrate import simps       #To integrate for each band
from itertools import combinations      #To generate every combinations possible

In [73]:
def PSD(x, windows = windows_MI, channels = channels):
    '''WARNING : If using scipy 1.14.0 or latest version, use simpson function instead of simps'''
    # PSD for each window of each channel
    psd, freqs = mne.time_frequency.psd_array_multitaper(x=x, sfreq=sfreq, bandwidth=0.5, low_bias=True, normalization='full', n_jobs=-1)
    
    #Every bands analysized [gamma, beta, alpha, theta, delta]
    bands = [(0.5, 4), (4, 8), (8, 12), (12, 35), (35, 60)]
    band_names = ["delta", "theta", "alpha", "beta", "gamma"]
    n = len(bands)

    #Creating matrix to store spectral_ratios
    spectral_ratios = np.zeros((windows, channels, n, n))
    spectral_powers = np.zeros((windows, channels, n))

    names_ratio = [[["" for i in range(5)] for j in range(5)] for k in range(channels)]
    names_power = []

    
    #Setting the maximum frequency in the measured bands
    fmin=bands[0][0]
    fmax=bands[-1][1]
    
    for i, (fmin, fmax) in enumerate(bands):
        
        #Freq indices in the band
        freq_idx = np.where((freqs >= fmin) & (freqs <= fmax))[0]

        #Frequency resolution
        freq_res = freq_idx[-1] - freq_idx[0]

        #Spectral power (we integrate behind the curve on the frequencies intervall of the band)
        spectral_power_band = simps(psd[:, :, freq_idx], dx=freq_res)

        #Spectral power of the band
        spectral_powers[:, :, i] = spectral_power_band

        #Dimensions of the power array
        x, y, z = spectral_powers.shape

    #Assigning names (channel 1 - all bands, channel 2 - all bands, ...)
    for j in range(channels):
        power_band_names = [f"Power channel {channel_names[j]} band {band_names[i]}" for i in range(n)]
        names_power.append(power_band_names)
    
    #We need to divide by the sum and not the total_power of the signal because simps is an approximation that does not work well on high resolution
    spectral_powers = spectral_powers / np.sum(spectral_powers, axis=-1)[:, :, np.newaxis]

    for i in range(n):
        for j in range(i+1, n):
            spectral_ratios[:, :, i, j] = spectral_powers[:, :, i] / spectral_powers[:, :, j]
            spectral_ratios[:, :, j, i] = spectral_powers[:, :, j] / spectral_powers[:, :, i]

    #Not optimal method since we re-parse the matrix ...
    #But it works 
    for k in range(channels):
        for i in range(n):
            for j in range(i+1, n):

                #Naming the ratios
                names_ratio[k][i][j] = f"Ratio channel {channel_names[k]} {band_names[i]}/{band_names[j]}"
                names_ratio[k][j][i] = f"Ratio channel {channel_names[k]} {band_names[j]}/{band_names[i]}"

    #Deleting the diagonales which are equal to zero
    spectral_ratios = spectral_ratios[np.nonzero(spectral_ratios)]
    
    columns = []
    #Flattening the list of lists into one dimension list
    for sublist in names_power:
        columns.extend(sublist)

    for sublist in names_ratio:
        for sub in sublist:
            columns.extend(sub)
    
    while "" in columns:
        columns.remove("")
    
    array = np.concatenate((spectral_powers.reshape((x, y*z)), spectral_ratios.reshape((x, y*(z**2 - z)))), axis=-1)
    X = pd.DataFrame(array, columns=columns)

    return X

In [294]:
#For acquisition
X_PSD_acquisition = PSD(data_array_acquisition, windows=windows_acquisition)

# (# of windows, 27*5*2)
print(X_PSD_acquisition.shape)

#Saving the df
save_as_csv(X_PSD_acquisition, 'Raw_PSD_acquisition_MI_BCI')

del X_PSD_acquisition

#For Motor Imagery
X_PSD_MI = PSD(data_array_MI)

# (# of windows, 27*5*2)
print(X_PSD_MI.shape)

#Saving the df
save_as_csv(X_PSD_MI, 'Raw_PSD_onlineT_MI_BCI')

del X_PSD_MI

#For baseline
X_PSD_baseline = PSD(data_array_baseline, windows=windows_baseline, channels=channels)

# (# of windows, 27*5*2)
print(X_PSD_baseline.shape)

#Saving the df
save_as_csv(X_PSD_baseline, 'Raw_PSD_baseline_MI_BCI')

del X_PSD_baseline

    Using multitaper spectrum estimation with 3 DPSS windows


C:\Users\gricih01\AppData\Local\Temp\ipykernel_8796\2427907505.py:32: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  spectral_power_band = simps(psd[:, :, freq_idx], dx=freq_res)


(4400, 675)
PSD_acquisition_MI_BCI saved as .csv
    Using multitaper spectrum estimation with 3 DPSS windows
(8800, 675)
PSD_onlineT_MI_BCI saved as .csv
    Using multitaper spectrum estimation with 1 DPSS windows
(4160, 675)
PSD_baseline_MI_BCI saved as .csv


#### Coherence feature

In [295]:
from scipy.signal import coherence

def Coherence(data, windows = windows_MI, channels = channels):

    total = ((channels**2-channels)/2)*5
    print(f"We will calculate {int(total)} coherence values")

    #Reminding of the band frequencies we're focused on
    bands = [(1, 4), (4, 8), (8, 12), (12, 35), (35, 80)]
    band_names = ["delta", "theta", "alpha", "beta", "gamma"]

    #Creating a matrix for the coherence

    n = int((channels**2-channels)/2)   #(channels**2 - channels )/ 2 equal the number of combinations of 2 channels without repetition
    coh = np.zeros((windows, n, len(bands)))
    coh_names = []


    #For each pair of electrodes
    comb = combinations(range(channels), 2)
    
    for c in comb:
        x, y = c
        #Calculate the coherence for every frequency found in the signal
        coh_freq, cohe = coherence(data[:, x, :], data[:, y, :], fs=sfreq)

        #Adjusting the index from (x, y) to z considering the triangular matrix of calculated coherence
        idx = x * (channels - 1) - (x * (x - 1)) // 2 + y - (x + 1)
        
        #Calculate coherence for each band
        for i, (fmin, fmax) in enumerate(bands):
            freq_idx = np.where((coh_freq >= fmin) & (coh_freq <= fmax))[0]
            
            #Here we calculate the coherence of the band by the mean but we could also use an integral
            coh_band = np.mean(cohe[:, freq_idx], axis=-1)
            
            coh[:, idx, i] = coh_band

            #Namming each feature so we can retrieve which ones are the best for classification
            coh_names.append(f"Coherence channels {channel_names[x]}-{channel_names[y]} band {band_names[i]}")
    
    array = coh.reshape(windows, n*len(bands))
    X = pd.DataFrame(array, columns=coh_names)

    return X

In [297]:
#Atleast 1hour execution time

#For acquisition
X_COH_acquisition = Coherence(data_array_acquisition, windows=windows_acquisition)

# (# of windows, # of comb * # of bands)
print(X_COH_acquisition.shape)

#Saving the df
save_as_csv(X_COH_acquisition, 'Raw_COH_acquisition_MI_BCI')

del X_COH_acquisition

#For Motor Imagery
X_COH_MI = Coherence(data_array_MI)

# (# of windows, # of comb * # of bands)
print(X_COH_MI.shape)

#Saving the df
save_as_csv(X_COH_MI, 'Raw_COH_onlineT_MI_BCI')

del X_COH_MI

#For baseline
X_COH_baseline = Coherence(data_array_baseline, windows=windows_baseline, channels=channels)

# (# of windows, # of comb * # of bands)
print(X_COH_baseline.shape)

#Saving the df
save_as_csv(X_COH_baseline, 'Raw_COH_baseline_MI_BCI')

del X_COH_baseline

We will calculate 1755 coherence values
(4400, 1755)
COH_acquisition_MI_BCI saved as .csv
We will calculate 1755 coherence values
(8800, 1755)
COH_onlineT_MI_BCI saved as .csv
We will calculate 1755 coherence values
(4160, 1755)
COH_baseline_MI_BCI saved as .csv


Function to avoid overfitting on the CSP features (we have to separate the 6 patterns for very fold)

In [298]:
def events_buckets(array, events, label, bins=5):
    # This function takes an epochs array, a label array and the integer bins and return the array buckets which store 5 bins (default) for every subject
    # (epochs, channels, resolution) * (epochs) * int -> (int, epochs//int, channels, resolution) * (int, epochs)

    classes = np.unique(label)

    x, y, z = array.shape
    buckets = np.zeros((bins, x//bins, y, z))

    events_labels = events[(events[:, 2] == 1) | (events[:, 2] == 2), -1]

    #Re-referencing the data since we merge them with other subjects
    buckets_events = np.zeros((bins, x//bins), dtype=int)

    #Separating for each subject
    for i in range(len(classes)):
        
        #Index of the epochs of the specified subject
        idx_epochs = np.where(label==classes[i])[0]
        n = len(idx_epochs)

        #Separating each subject into 5 bins
        for j in range(bins):
            index = idx_epochs[n//bins * j : n//bins * (j+1)]
            l = len(index)            
            buckets[j, l*i:l*(i+1), :, :] = array[index, :]
            buckets_events[j, l*i:l*(i+1)] = events_labels[index]

    return buckets, buckets_events

#### CSP feature

In [440]:
def CSP(x, events, labels, bins=5):

    buckets, buckets_events = events_buckets(x, events, labels, bins=bins)
    DF = []

    for i in tqdm(range(bins)):

        # Separating into bins-1 buckets as training
        index = [k for k in range(bins) if k != i]
        X = np.concatenate(buckets[index])
        y = np.concatenate(buckets_events[index])
        
        #Taking the last fold as the testing one
        X_test = buckets[i]

        # CSP calculation
        csp = mne.decoding.CSP(n_components=6, reg=0.01, log=True, norm_trace=True, rank={'eeg': 27, 'mag':27})
        
        # Fit CSP on epochs data
        csp.fit(X, y)

        # Apply CSP to epochs
        epochs_csp = csp.transform(X_test)

        columns = [f"CSP Pattern n°{k+1}" for k in range(epochs_csp.shape[1])]

        df = pd.DataFrame(epochs_csp, columns=columns)

        DF.append(df)

    return pd.concat(DF, axis=0)

In [441]:
#For acquisition
X_CSP_acquisition = CSP(data_array_acquisition, events_acquisition, label_array_acquisition)

# (# windows, 6)
print(X_CSP_acquisition.shape)

#Saving the df
save_as_csv(X_CSP_acquisition, 'Raw_CSP_acquisition_MI_BCI')

#To save memory
del X_CSP_acquisition


#For Motor Imagery
X_CSP_MI= CSP(data_array_MI, events_MI, label_array_MI)

# (# windows, 6)
print(X_CSP_MI.shape)

#Saving the df
save_as_csv(X_CSP_MI, 'Raw_CSP_onlineT_MI_BCI')

#Saving memory
del X_CSP_MI

  0%|          | 0/5 [00:00<?, ?it/s]

Computing rank from data with rank={'eeg': 27, 'mag': 27}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 27 -> 27
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 27, 'mag': 27}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 27 -> 27
Estimating covariance using SHRINKAGE
Done.


 20%|██        | 1/5 [00:43<02:54, 43.60s/it]

Computing rank from data with rank={'eeg': 27, 'mag': 27}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 27 -> 27
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 27, 'mag': 27}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 27 -> 27
Estimating covariance using SHRINKAGE
Done.


 40%|████      | 2/5 [01:04<01:31, 30.41s/it]

...

Computing rank from data with rank={'eeg': 27, 'mag': 27}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 27 -> 27
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 27, 'mag': 27}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 27 -> 27
Estimating covariance using SHRINKAGE
Done.


 80%|████████  | 4/5 [1:25:32<14:58, 898.70s/it] 

Computing rank from data with rank={'eeg': 27, 'mag': 27}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 27 -> 27
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 27, 'mag': 27}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 27 -> 27
Estimating covariance using SHRINKAGE
Done.


100%|██████████| 5/5 [1:30:23<00:00, 1084.79s/it]


(8640, 6)
CSP_onlineT_MI_BCI saved as .csv


#### Inter-hemispheric amplitude ratio (IHAR)

In [442]:
#Function to compute amplitude on the left/right electrodes
def compute_amplitude_rms(signal):
    return np.sqrt(np.mean(np.square(signal), axis=-1))

In [443]:
def IHAR(x, n=11):
    #x = (epochs, channels, resolution)
    #n = number of bilateral pair you want to select
    info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types='eeg')
    epochs = mne.EpochsArray(x, info)
    
    left = ['F3', 'FC5', 'FC3', 'FC1', 'C5', 'C3', 'C1', 'CP5', 'CP3', 'CP1', 'P3']  #Left  hemisphere
    right= ['F4', 'FC2', 'FC4', 'FC6', 'C2', 'C4', 'C6', 'CP2', 'CP4', 'CP6', 'P4']  #Right hemisphere

    #Computing the rms amplitudes for every left channel
    left_amplitude = epochs.copy().pick_channels(left[:n]).get_data()
    left_amplitude = compute_amplitude_rms(left_amplitude)

    #Same for right channels
    right_amplitude = epochs.copy().pick_channels(right[:n]).get_data()
    right_amplitude = compute_amplitude_rms(right_amplitude)
    
    IHAR_values = left_amplitude / right_amplitude
    
    columns = [f"IHAR : {left[i]}/{right[i]}" for i in range(n)]
    X = pd.DataFrame(IHAR_values, columns=columns)

    return X

In [444]:
#For acquisition
X_IHAR_acquisition = IHAR(data_array_acquisition)

# (# of windows, 1)
print(X_IHAR_acquisition.shape)

#Saving the df
save_as_csv(X_IHAR_acquisition, 'Raw_IHAR_acquisition_MI_BCI')

del X_IHAR_acquisition


#For Motor Imagery
X_IHAR_MI = IHAR(data_array_MI)

# (# of windows, 1)
print(X_IHAR_MI.shape)

#Saving the df
save_as_csv(X_IHAR_MI, 'Raw_IHAR_onlineT_MI_BCI')

del X_IHAR_MI


#For baseline
X_IHAR_baseline = IHAR(data_array_baseline)

# (# of windows, 1)
print(X_IHAR_baseline.shape)

#Saving the df
save_as_csv(X_IHAR_baseline, 'Raw_IHAR_baseline_MI_BCI')

del X_IHAR_baseline

Not setting metadata
4320 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\gricih01\AppData\Local\Temp\ipykernel_15276\1680328372.py:11: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  left_amplitude = epochs.copy().pick_channels(left[:n]).get_data()
C:\Users\gricih01\AppData\Local\Temp\ipykernel_15276\1680328372.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  left_amplitude = epochs.copy().pick_channels(left[:n]).get_data()


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\gricih01\AppData\Local\Temp\ipykernel_15276\1680328372.py:15: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  right_amplitude = epochs.copy().pick_channels(right[:n]).get_data()
C:\Users\gricih01\AppData\Local\Temp\ipykernel_15276\1680328372.py:15: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  right_amplitude = epochs.copy().pick_channels(right[:n]).get_data()


(4320, 11)
IHAR_acquisition_MI_BCI saved as .csv
Not setting metadata
8640 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

(8640, 11)
IHAR_onlineT_MI_BCI saved as .csv
Not setting metadata
4160 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\gricih01\AppData\Local\Temp\ipykernel_15276\1680328372.py:11: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  left_amplitude = epochs.copy().pick_channels(left[:n]).get_data()
C:\Users\gricih01\AppData\Local\Temp\ipykernel_15276\1680328372.py:11: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  left_amplitude = epochs.copy().pick_channels(left[:n]).get_data()


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\gricih01\AppData\Local\Temp\ipykernel_15276\1680328372.py:15: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  right_amplitude = epochs.copy().pick_channels(right[:n]).get_data()
C:\Users\gricih01\AppData\Local\Temp\ipykernel_15276\1680328372.py:15: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  right_amplitude = epochs.copy().pick_channels(right[:n]).get_data()


(4160, 11)
IHAR_baseline_MI_BCI saved as .csv
